In [ ]:
#Load main dependencies
import numpy as np
# import math
import random
import smtplib
import os
import pandas as pd
from datetime import datetime

#Load document modules 
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.text import WD_LINE_SPACING
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.enum.style import WD_STYLE_TYPE
from docx.shared import Pt

#Load email modules 
from os.path import basename
from time import sleep
from email import message
from email.message import EmailMessage
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

In [ ]:
# prefs = Table().read_table("Phoenix Fest Audience Sign Up 2022 (Responses) - Form Responses 1.csv")
prefs = pd.read_csv("Phoenix Fest Audience Sign Up 2023 (Responses).csv")
program = pd.read_csv("PF Program.csv")
presenters = pd.read_csv("Phoenix Fest Presenters 2023.csv")
roster = pd.read_csv("Total Student List.csv")


In [ ]:
prefs


In [ ]:
program


In [ ]:
presenters


In [ ]:
presenters = presenters
presenters = presenters.rename(columns={"E-mail 1 - Value": "Email Address"}).drop(
    columns=["Additional Name"]
)
presenters


In [ ]:
roster = pd.read_csv("Total Student List.csv")
# Legacy code that was needed for a version of the student roster that was incorrect. Might be useful. 

# Remove all whitespace from the name 
# student_names = np.char.strip(roster["Student name"].to_numpy(dtype=str))

# first_names = np.empty(0)
# last_names = np.empty(0)

# Function to remove all the commas in the name
# def strip(val):
#     return val.replace(",", "")

# Strip every name and assign the first two values in the stripped list to the last and fist name, appending it to the appropriate array
# for name in student_names:
#     name = list(map(strip, name.split(" ")))
#     last, first = name[0:2]
#     first_names = np.append(first_names, first)
#     last_names = np.append(last_names, last)

# Add these columns to the roster table
# roster.insert(1, "First Name", first_names, True)
# roster.insert(2, "Last Name", last_names, True)


roster = roster.rename(
    columns={
        "First name": "First Name",
        "Last name": "Last Name",
        "Grade": "Grade Level",
    }
)
roster


In [ ]:
# The following cell finds all of the people that didn't sign up
# These lines are totally unnecessary unless you run this cell multiple times; just saves me a headache
prefs = pd.read_csv("Phoenix Fest Audience Sign Up 2023 (Responses).csv")
roster = roster

roster["Email"] = np.char.lower(roster["Email"].to_numpy(dtype=str))
good_to_go = np.empty(0)
signed_up = np.char.lower(prefs["Email Address"].to_numpy(dtype=str))
presenters_emails = np.char.lower(presenters["Email Address"].to_numpy(dtype=str))
print(f"Presenters that signed up: {np.intersect1d(presenters_emails,signed_up)}")

""" 
The point of the following line  is to consolidate all of the presenters and those that did sign up. However,
we can always assume that some presenters did sign-up by mistake, so we have to use np.unique() to remove them
"""
good_to_go = np.unique(np.append(signed_up, presenters_emails))
no_sign_up = np.setdiff1d(roster["Email"], good_to_go)
nan_table = pd.DataFrame(no_sign_up, columns=["Email Address"])


print(f"NUMBER OF PEOPLE TO ASSIGN: {len(roster.index) - len(presenters_emails)}")
actual_signed_up = pd.DataFrame(
    np.setdiff1d(signed_up, presenters_emails), columns=["Email Address"]
)
prefs = pd.merge(actual_signed_up, prefs)
move_timestamp = prefs.pop("Timestamp")
prefs.insert(0, "Timestamp", move_timestamp)
nan_table = pd.merge(nan_table, roster, left_on="Email Address", right_on="Email").drop(
    columns=["Email"]
)
nan_table


In [ ]:
def session_assign(prefs_table, program_table, event_name, choice, block):
    # Grabbing all of the people who wanted the session
    event_table = prefs_table.loc[
        prefs_table[choice] == event_name,
        [
            "Email Address",
            "What is your LAST name?",
            "What is your FIRST name?",
            "What grade are you in?",
            choice,
        ],
    ].sample(frac=1)

    
    roster = np.empty(0)
    extra = None

    # Max cap from programs table; have to filter by category first for programs in multiple blocks
    max_cap = program.loc[
        (program_table["Category"] == block)
        & (program_table["Program Title"] == event_name),
        "Max Cap",
    ].item()

    # Min cap from programs table
    min_cap = program.loc[
        (program_table["Category"] == block)
        & (program_table["Program Title"] == event_name),
        "Min Cap",
    ].item()

    if len(event_table.index) <= max_cap:
        # Just assign everyone who wanted the session if enough spots are available
        roster = np.append(roster, event_table["Email Address"])
    else:
        # If there are too many people to assign, just assign as much as possible and save the rest to extra
        roster = event_table[:max_cap]["Email Address"].to_numpy(dtype=str)
        extra = event_table[max_cap : len(event_table.index)]["Email Address"].to_numpy(
            dtype=str
        )
    
    # Return final table
    data = {
        "Event Name": event_name,
        "People": [roster],
        "Min Cap": min_cap,
        "Max Cap": max_cap,
        "Attendees": len(roster),
        "Spots Left": max_cap - len(event_table.index),
        "Extras": [extra],
    }
    return pd.DataFrame(data)


In [ ]:
def schedule(prefs_table, program_table, choice, category):
    session_events = program_table.loc[program_table["Category"] == category][
        "Program Title"
    ].to_numpy(
        dtype=str
    )  # Grab all the programs from the programs spreadsheet

    # Create the main arrays/lists. These will contain all of the information from all of the sessions
    people = []
    max_cap = np.empty(0)
    min_cap = np.empty(0)
    attendees = np.empty(0)
    aval_spots = []
    extras = []
    reassign = np.empty(0)

    # Find all of the people who for some reason did not select a first choice for this session
    removed_sign_up = prefs_table.loc[prefs_table[choice] == "NONE"][
        "Email Address"
    ].to_numpy(dtype=str)
    for x in np.arange(0, len(removed_sign_up)):
        reassign = np.append(reassign, removed_sign_up[x])
    # print(f"{category}: {reassign}")

    for event in session_events:
        session_table = session_assign(
            prefs_table, program_table, event, choice, category
        )
        people.append(session_table["People"][0])
        max_cap = np.append(max_cap, session_table["Max Cap"].item())
        min_cap = np.append(min_cap, session_table["Min Cap"].item())
        attendees = np.append(attendees, session_table["Attendees"].item())
        aval_spots = np.append(aval_spots, session_table["Spots Left"].item())
        extras.append(session_table["Extras"][0])  # Grab the extras list

    # Return complete schedule
    data = {
        "Program Title": session_events,
        "People": people,
        "Min Cap": min_cap,
        "Max Cap": max_cap,
        "Attendees": attendees,
        "Spots Left": aval_spots,
        "Extras": extras,
    }
    return pd.DataFrame(data), reassign


In [ ]:
def reschedule(table, choice, reassign=None):
    peoples = []
    
    for people in table.loc[table["Spots Left"] < 0]["Extras"].to_numpy():
        # Grabbing all of the people in the extras colum
        for num in np.arange(0, len(people)):
            peoples.append(people[num])
    if isinstance(reassign, np.ndarray):
        # Checking to see if there is anyone in the reassign array; if so, add them to the extras array to assign
        np.array(peoples)
        peoples = np.append(peoples, reassign)
    if len(peoples) == 0:
        # Reassign exception because possibly everyone has already be assigned
        return table, peoples

    # For all the people that need to be reassigned, grab their preferences and shuffle them up 
    peoples = pd.DataFrame(peoples, columns=["People"])
    peoples = (
        pd.merge(peoples, prefs, left_on="People", right_on="Email Address")
        .drop(columns=["People"])
        .sample(frac=1)
    )

    #Find all of the remaining sessions that have spots available
    aval_sessions = table.loc[table["Spots Left"] > 0]
    aval_sessions_list = aval_sessions["Program Title"].to_numpy(dtype=str)

    # Main arrays/lists
    reassign = np.empty(0)
    new_attendees = []
    attendees_numbers = np.empty(0)
    new_spots = np.empty(0)
    new_extras = []

    for person in peoples["Email Address"]:
        # Checking if the desired session is already full; if it is, then assign them to new reassign array
        session_pref = peoples.loc[peoples["Email Address"].to_numpy() == person][
            choice
        ].item()
        if session_pref not in aval_sessions_list:
            reassign = np.append(reassign, person)

    if len(np.setdiff1d(peoples["Email Address"].to_numpy(dtype=str), reassign)) == 0:
        # Reassign exception: An Instance all of the desired sessions are full
        return table, peoples["Email Address"].to_numpy(dtype=str)
        # raise ValueError("Can't assign; everyone that can be assigned has been")

    # Removing everyone in the reassign array from the peoples array
    assign_table = pd.DataFrame(
        np.setdiff1d(peoples["Email Address"].to_numpy(dtype=str), reassign),
        columns=["People"],
    )
    assign_table = pd.merge(
        assign_table, prefs, left_on="People", right_on="Email Address"
    ).drop(columns=["People"])

    for event in aval_sessions_list:
        event_people = assign_table.loc[assign_table[choice] == event]
        attendees = aval_sessions.loc[aval_sessions["Program Title"] == event][
            "People"
        ].item()  # Grab all of the people currently in the session
        spots_left = aval_sessions.loc[aval_sessions["Program Title"] == event][
            "Spots Left"
        ].item()  # Current amount of spots left

        # Very similar set of code to session_assign; if there is enough spots, assign everyone!
        if len(event_people.index) <= spots_left:
            attendees = np.append(
                attendees, event_people["Email Address"].to_numpy(dtype=str)
            )
            new_attendees.append(attendees)
            attendees_numbers = np.append(attendees_numbers, len(attendees))
            new_spots_left = spots_left - len(event_people.index)
            new_spots = np.append(new_spots, new_spots_left)
            new_extras.append(None)
        else:
            attendees = np.append(
                attendees,
                event_people[: int(spots_left)]["Email Address"].to_numpy(dtype=str),
            )
            new_attendees.append(attendees)
            attendees_numbers = np.append(attendees_numbers, len(attendees))
            new_spots = np.append(new_spots, spots_left - len(event_people.index))
            new_extras.append(
                event_people[int(spots_left) : len(event_people.index)][
                    "Email Address"
                ].to_numpy(dtype=str)
            )

    # Return Final Rescheduled Table
    data = {
        "Program Title": aval_sessions_list,
        "People": new_attendees,
        "Min Cap": aval_sessions["Min Cap"],
        "Max Cap": aval_sessions["Max Cap"],
        "Attendees": attendees_numbers,
        "Spots Left": new_spots,
        "Extras": new_extras,
    }
    return pd.DataFrame(data), reassign


In [ ]:
def final_schedule(table, reassign, nan_table):
    """
    The goal of this function is to simply resolve any remaining people that don't have a session for the bloc
    and any people that did not sign up for an event (nan_table). It assigns as many people as possible to as many spots
    possible, starting with the sessions with the highest number of spots left.
    """
    # Throw an error if all people are in a session already (highly unlikely!)
    if len(table.index) == 0:
        raise ValueError(
            "Can't do final schedule! All of the sessions are full already :)"
        )
    people = np.empty(0)
    for person in table.loc[table["Spots Left"] < 0]["Extras"].to_numpy():
        people = np.append(people, person)

    # Combine those who did not sign up with those in the "Extras" column
    people = np.append(people, nan_table["Email Address"])

    # Compile everyone together, including those whose 5th choice was unavailable (This is now everyone!)
    people = np.append(people, reassign)

    random.shuffle(people)  # Shuffle everyone up!

    # Remove all of the spots that are full from the 5th assign
    table = table.loc[table["Spots Left"] > 0].sort_values(
        by="Spots Left", ascending=False
    )
    new_people = []
    new_attendees_count = np.empty(0)

    for i in np.arange(0, len(table.index)):
        # Grab all of the current data about the session (prior to any assigning)
        all_attendees = np.empty(0)
        current_attendees = table.iloc[i]["People"]
        spots_left = int(table.iloc[i]["Spots Left"].item())
        current_attendees_count = int(table.iloc[i]["Attendees"])

        # The following lines of code handles the very final session, when there are more spots left than people left to assign

        if len(people) <= spots_left:
            if len(people) != 0:
                """Two parts here: This is the first one if there are still a few people
                that need to be assigned, just assign them to this session"""
                new_attendees = people
                new_attendees_count = np.append(
                    new_attendees_count, (current_attendees_count + len(new_attendees))
                )
                all_attendees = np.append(
                    all_attendees, np.append(new_attendees, current_attendees)
                )
                new_people.append(all_attendees)

                """ 
                Probably the most confusing lines of code in the entire program. But, the gist here is that if there are no 
                more people left (len(people) == 0), we need to include that session in grabbing data for the sessions with no 
                new attendees, otherwise just grab all of the ones after this (hence, why lines of code above break the loop if we are on the
                second-to-last or last session)
                """

                # This line handles if this is the last or second to last session
                if i == len(table.index) - 1 or i == len(table.index):
                    break

            if len(people) == 0:
                k = 0
            else:
                k = 1

            for j in np.arange((i + k), len(table.index)):
                """ Second part, just grab the data any potential sessions with no new attendees  """
                current_attendees = table.iloc[j]["People"]
                current_attendees_count = int(table.iloc[j]["Attendees"])
                new_people.append(current_attendees)
                new_attendees_count = np.append(
                    new_attendees_count, current_attendees_count
                )
            break

        # This line of code is the "normal" procedure when there are more people than available spots for a given session; same session_assign() logic we have seen
        new_attendees = people[np.arange(0, spots_left)]
        all_attendees = np.append(
            all_attendees, np.append(new_attendees, current_attendees)
        )
        new_people.append(all_attendees)
        new_attendees_count = np.append(
            new_attendees_count, (current_attendees_count + len(new_attendees))
        )
        # Remove all of the people who were just assigned from the 'peoples' array
        people = np.delete(people, np.arange(0, spots_left))
    new_spots_left = table["Max Cap"].to_numpy() - new_attendees_count

    data = {
        "Program Title": table["Program Title"].to_numpy(),
        "People": new_people,
        "Min Cap": table["Min Cap"].to_numpy(),
        "Max Cap": table["Max Cap"].to_numpy(),
        "Attendees": new_attendees_count,
        "Spots Left": new_spots_left,
    }

    return pd.DataFrame(data)


In [ ]:
def remove_numbers(with_numbers):
    new_program_titles = np.empty(0)
    for program_title in with_numbers:
        if program_title == "NONE":
            continue
        split_title = program_title.split(".")
        new_program_titles = np.append(new_program_titles, split_title[1].strip())
    return new_program_titles


In [ ]:
def combine(
    table_one, table_two, table_three, table_four, table_five, final_table, category
):
    """
    Combine is the main session function that combines all of rosters generated from the various
    schedule and reschedule functions and combine them into one complete table for the bloc. It requires
    6 tables, which include the initial schedule table, the 4 reschedules and the final table from final_schedule()
    """
    # Main arrays/lists
    programs_array = np.empty(0)
    peoples = []
    min_cap = np.empty(0)
    max_cap = np.empty(0)
    attendees = np.empty(0)
    spots_left = np.empty(0)

    sessions_one = table_one["Program Title"].to_numpy(dtype=str)
    sessions_two = table_two["Program Title"].to_numpy(dtype=str)
    sessions_three = table_three["Program Title"].to_numpy(dtype=str)
    sessions_four = table_four["Program Title"].to_numpy(dtype=str)
    sessions_five = table_five["Program Title"].to_numpy(dtype=str)
    sessions_final = final_table["Program Title"].to_numpy(dtype=str)

    # Finding the difference between the tables of the two sessions, grab essentially gets the info for only the sessions that filled during a specific assign
    def grab(session_one, session_two, table):
        nonlocal programs_array, peoples, min_cap, max_cap, attendees, spots_left
        for event in np.setdiff1d(session_one, session_two):
            programs_array = np.append(programs_array, event)
            people = table.loc[table["Program Title"] == event]["People"].to_numpy()[0]
            peoples.append(people)
            min_cap = np.append(
                min_cap, table.loc[table["Program Title"] == event]["Min Cap"].item()
            )
            max_cap = np.append(
                max_cap, table.loc[table["Program Title"] == event]["Max Cap"].item()
            )
            attendees = np.append(
                attendees,
                table.loc[table["Program Title"] == event]["Attendees"].item(),
            )
            spots_left = np.append(
                spots_left,
                table.loc[table["Program Title"] == event]["Spots Left"].item(),
            )

    grab(sessions_one, sessions_two, table_one)
    grab(sessions_two, sessions_three, table_two)
    grab(sessions_three, sessions_four, table_three)
    grab(sessions_four, sessions_five, table_four)
    grab(sessions_five, sessions_final, table_five)
    # Use an empty list to get all of the sessions that aren't fill yet!
    grab(sessions_final, [], final_table)

    data = {
        "Program Title": programs_array,
        "People": peoples,
        "Min Cap": min_cap,
        "Max Cap": max_cap,
        "Attendees": attendees,
        "Spots Left": spots_left,
    }

    df = pd.DataFrame(data)

    df = df.sort_values(by="Program Title")

    df["Room Location"] = (
        program.loc[program["Category"] == category]
        .sort_values(by="Program Title")["Room Teacher"]
        .to_numpy(dtype=str)
    )

    df["Supervisor"] = (
        program.loc[program["Category"] == category]
        .sort_values(by="Program Title")["Supervisor"]
        .to_numpy(dtype=str)
    )

    df["Category"] = category

    df["Room #"] = (
        program.loc[program["Category"] == category]
        .sort_values(by="Program Title")["Room #"]
        .to_numpy(dtype=str)
    )

    # Remove the numbers from the program titles
    new_program_titles = remove_numbers(df["Program Title"].to_numpy(dtype=str))

    df["Program Title"] = new_program_titles

    df = df.sort_values(by="Program Title")

    return df


In [ ]:
# Testing Block!

# test = schedule(prefs, program, "Which presentation is your first choice for the Academic block?", 1)
# test_two = reschedule(test[0],"Which presentation is your second choice for the Academic block?", test[1])
# test_three = reschedule(test_two[0],"Which presentation is your third choice for the Academic block?", test_two[1])
# test_four = reschedule(test_three[0],"Which presentation is your fourth choice for the Academic block?", test_three[1])
# test_five = reschedule(test_four[0],"Which presentation is your fifth choice for the Academic block?", test_four[1])
# test_final = final_schedule(test_five[0], test_five[1], nan_table)
# combine_final = combine(test[0], test_two[0], test_three[0], test_four[0], test_five[0], test_final, 1)

# test_six = schedule(prefs, program, "Which presentation is your first choice for the Culture block?", 2)
# test_seven = reschedule(test_six[0],"Which presentation is your second choice for the Culture block?", test_six[1])
# test_eight = reschedule(test_seven[0],"Which presentation is your third choice for the Culture block?", test_seven[1])
# test_nine = reschedule(test_eight[0], "Which presentation is your fourth choice for the Culture block?", test_eight[1])
# test_ten = reschedule(test_nine[0],"Which presentation is your fifth choice for the Culture block?", test_nine[1])
# final_two = final_schedule(test_ten[0], test_ten[1], nan_table)
# combine_two = combine(test_six[0], test_seven[0], test_eight[0], test_nine[0], test_ten[0], final_two, 2)

# test_eleven = schedule(prefs, program, "Which presentation is your first choice for the FEST block?", 3)
# test_twelve = reschedule(test_eleven[0],"Which presentation is your second choice for the Fest block?", test_eleven[1])
# test_thirteen = reschedule(test_twelve[0],"Which presentation is your third choice for the Fest block?", test_twelve[1])
# test_fourteen = reschedule(test_thirteen[0],"Which presentation is your fourth choice for the Fest block?", test_thirteen[1])
# test_fifteen = reschedule(test_fourteen[0],"Which presentation is your fifth choice for the Fest block?", test_fourteen[1])
# final_three = final_schedule(test_fifteen[0], test_fifteen[1], nan_table)
# combine_three = combine(test_eleven[0], test_twelve[0], test_thirteen[0], test_fourteen[0], test_fifteen[0], final_three, 3)
# combine_three

# test_sixteen = schedule(prefs, program, "Which presentation is your first choice for the Community block?", 4)
# test_seventeen = reschedule(test_sixteen[0],"Which presentation is your second choice for the Community block?", test_sixteen[1])
# test_eighteen = reschedule(test_seventeen[0],"Which presentation is your third choice for the Community block?", test_seventeen[1])
# test_nineteen = reschedule(test_eighteen[0],"Which presentation is your fourth choice for the Community block?", test_eighteen[1])
# test_twenty = reschedule(test_nineteen[0],"Which presentation is your fifth choice for the Community block?", test_nineteen[1])
# final_four = final_schedule(test_twenty[0], test_twenty[1], nan_table)
# combine_four = combine(test_sixteen[0], test_seventeen[0], test_eighteen[0], test_nineteen[0], test_twenty[0], final_four, 4)
# combine_four

# test_twenty1 = schedule(prefs, program, "Which presentation is your first choice for the Experience block?", 5)
# test_twenty2 = reschedule(test_twenty1[0],"Which presentation is your second choice for the Experience block?", test_twenty1[1])
# test_twenty3 = reschedule(test_twenty2[0],"Which presentation is your third choice for the Experience block?", test_twenty2[1])
# test_twenty4 = reschedule(test_twenty3[0],"Which presentation is your fourth choice for the Experience block?", test_twenty3[1])
# test_twenty5 = reschedule(test_twenty4[0],"Which presentation is your fifth choice for the Experience block?", test_twenty4[1])
# final_five = final_schedule(test_twenty5[0], test_twenty5[1], nan_table)
# combine_five = combine(test_twenty1[0], test_twenty2[0], test_twenty3[0], test_twenty4[0], test_twenty5[0], final_five, 5)
# combine_five


In [ ]:
def combine_session(one, two, three, four, five):
    """
    Combine sessions is very similar to functionality to combine(), but it just does so for the entire Phoenix Fest program,
    combing all 5 blocs into one table that can be exported. It requires 5 tables, which are the tables generated in each session
    """
    program_titles = np.empty(0)
    people = []
    min_caps = np.empty(0)
    max_caps = np.empty(0)
    attendees = np.empty(0)
    spots_left = np.empty(0)
    category = np.empty(0)
    rooms = np.empty(0)
    room_locations = np.empty(0)
    supervisors = np.empty(0)

    def unpack(session):
        nonlocal program_titles, people, min_caps, max_caps, attendees, spots_left, category, rooms, room_locations, supervisors
        program_titles = np.append(
            program_titles, session["Program Title"].to_numpy(dtype=str)
        )
        for i in np.arange(0, len(session.index)):
            people.append(session["People"].to_numpy()[i])
        min_caps = np.append(min_caps, session["Min Cap"].to_numpy(dtype=str))
        max_caps = np.append(max_caps, session["Max Cap"].to_numpy(dtype=str))
        attendees = np.append(attendees, session["Attendees"].to_numpy(dtype=str))
        spots_left = np.append(spots_left, session["Spots Left"].to_numpy(dtype=str))
        category = np.append(category, session["Category"].to_numpy(dtype=str))
        rooms = np.append(rooms, session["Room #"].to_numpy(dtype=str))
        room_locations = np.append(
            room_locations, session["Room Location"].to_numpy(dtype=str)
        )
        supervisors = np.append(supervisors, session["Supervisor"].to_numpy(dtype=str))

    unpack(one)
    unpack(two)
    unpack(three)
    unpack(four)
    unpack(five)

    data = {
        "Program Title": program_titles,
        "People": people,
        "Min Cap": min_caps,
        "Max Cap": max_caps,
        "Attendees": attendees,
        "Spots Left": spots_left,
        "Room Location": room_locations,
        "Supervisor": supervisors,
        "Category": category,
        "Room #": rooms,
    }

    return pd.DataFrame(data)


In [ ]:
def people_schedule(programs_session_table):
    """
    People Schedule takes the combined schedule for a specific block and displays what each person has for that session.
    This makes the later process of creating individual schedules much easier.
    """
    people = np.empty(0)
    program = np.empty(0)
    room_numbers = np.empty(0)
    room_locations = np.empty(0)

    for i in np.arange(0, len(programs_session_table.index)):
        row = programs_session_table.iloc[i]
        for x in np.arange(0, len(row["People"])):
            people = np.append(people, row["People"][x])
            program = np.append(program, row["Program Title"])
            room_numbers = np.append(room_numbers, row["Room #"])
            room_locations = np.append(room_locations, row["Room Location"])

    data = {
        "People": people,
        "Program": program,
        "Room #": room_numbers,
        "Room Location": room_locations,
    }

    return pd.DataFrame(data)


In [ ]:
def session_one():
    schedule_one = schedule(
        prefs,
        program,
        "Which presentation is your first choice for the Academic block?",
        1,
    )
    schedule_two = reschedule(
        schedule_one[0],
        "Which presentation is your second choice for the Academic block?",
        schedule_one[1],
    )
    schedule_three = reschedule(
        schedule_two[0],
        "Which presentation is your third choice for the Academic block?",
        schedule_two[1],
    )
    schedule_four = reschedule(
        schedule_three[0],
        "Which presentation is your fourth choice for the Academic block?",
        schedule_three[1],
    )
    schedule_five = reschedule(
        schedule_four[0],
        "Which presentation is your fifth choice for the Academic block?",
        schedule_four[1],
    )
    final = final_schedule(schedule_five[0], schedule_five[1], nan_table)
    sessions = combine(
        schedule_one[0],
        schedule_two[0],
        schedule_three[0],
        schedule_four[0],
        schedule_five[0],
        final,
        1,
    )
    people = people_schedule(sessions)
    return sessions, people


In [ ]:
def session_two():
    schedule_one = schedule(
        prefs,
        program,
        "Which presentation is your first choice for the Culture block?",
        2,
    )
    schedule_two = reschedule(
        schedule_one[0],
        "Which presentation is your second choice for the Culture block?",
        schedule_one[1],
    )
    schedule_three = reschedule(
        schedule_two[0],
        "Which presentation is your third choice for the Culture block?",
        schedule_two[1],
    )
    schedule_four = reschedule(
        schedule_three[0],
        "Which presentation is your fourth choice for the Culture block?",
        schedule_three[1],
    )
    schedule_five = reschedule(
        schedule_four[0],
        "Which presentation is your fifth choice for the Culture block?",
        schedule_four[1],
    )
    final = final_schedule(schedule_five[0], schedule_five[1], nan_table)
    sessions = combine(
        schedule_one[0],
        schedule_two[0],
        schedule_three[0],
        schedule_four[0],
        schedule_five[0],
        final,
        2,
    )
    people = people_schedule(sessions)
    return sessions, people


In [ ]:
def session_three():
    schedule_one = schedule(
        prefs, program, "Which presentation is your first choice for the FEST block?", 3
    )
    schedule_two = reschedule(
        schedule_one[0],
        "Which presentation is your second choice for the Fest block?",
        schedule_one[1],
    )
    schedule_three = reschedule(
        schedule_two[0],
        "Which presentation is your third choice for the Fest block?",
        schedule_two[1],
    )
    schedule_four = reschedule(
        schedule_three[0],
        "Which presentation is your fourth choice for the Fest block?",
        schedule_three[1],
    )
    schedule_five = reschedule(
        schedule_four[0],
        "Which presentation is your fifth choice for the Fest block?",
        schedule_four[1],
    )
    final = final_schedule(schedule_five[0], schedule_five[1], nan_table)
    sessions = combine(
        schedule_one[0],
        schedule_two[0],
        schedule_three[0],
        schedule_four[0],
        schedule_five[0],
        final,
        3,
    )
    people = people_schedule(sessions)
    return sessions, people


In [ ]:
def session_four():
    schedule_one = schedule(
        prefs,
        program,
        "Which presentation is your first choice for the Community block?",
        4,
    )
    schedule_two = reschedule(
        schedule_one[0],
        "Which presentation is your second choice for the Community block?",
        schedule_one[1],
    )
    schedule_three = reschedule(
        schedule_two[0],
        "Which presentation is your third choice for the Community block?",
        schedule_two[1],
    )
    schedule_four = reschedule(
        schedule_three[0],
        "Which presentation is your fourth choice for the Community block?",
        schedule_three[1],
    )
    schedule_five = reschedule(
        schedule_four[0],
        "Which presentation is your fifth choice for the Community block?",
        schedule_four[1],
    )
    final = final_schedule(schedule_five[0], schedule_five[1], nan_table)
    sessions = combine(
        schedule_one[0],
        schedule_two[0],
        schedule_three[0],
        schedule_four[0],
        schedule_five[0],
        final,
        4,
    )
    people = people_schedule(sessions)
    return sessions, people


In [ ]:
def session_five():
    schedule_one = schedule(
        prefs,
        program,
        "Which presentation is your first choice for the Experience block?",
        5,
    )
    schedule_two = reschedule(
        schedule_one[0],
        "Which presentation is your second choice for the Experience block?",
        schedule_one[1],
    )
    schedule_three = reschedule(
        schedule_two[0],
        "Which presentation is your third choice for the Experience block?",
        schedule_two[1],
    )
    schedule_four = reschedule(
        schedule_three[0],
        "Which presentation is your fourth choice for the Experience block?",
        schedule_three[1],
    )
    schedule_five = reschedule(
        schedule_four[0],
        "Which presentation is your fifth choice for the Experience block?",
        schedule_four[1],
    )
    final = final_schedule(schedule_five[0], schedule_five[1], nan_table)
    sessions = combine(
        schedule_one[0],
        schedule_two[0],
        schedule_three[0],
        schedule_four[0],
        schedule_five[0],
        final,
        5,
    )
    people = people_schedule(sessions)
    return sessions, people


In [ ]:
def check_sessions(assigned_session, chosen_sessions_list):
    chosen_sessions_list = list(chosen_sessions_list)
    if assigned_session in chosen_sessions_list:
        return chosen_sessions_list.index(assigned_session) + 1
    else:
        return 6


def one_score(email, final_choices_table):
    if email in nan_table["Email Address"].to_numpy(dtype=str):
        return 30
    score = 0
    sessions = final_choices_table.loc[final_choices_table["Email"] == email][
        ["Session One", "Session Two", "Session Three", "Session Four", "Session Five"]
    ].iloc[0]
    choices = [
        prefs.loc[prefs["Email Address"] == email].iloc[0][6:11].to_numpy(dtype=str),
        prefs.loc[prefs["Email Address"] == email].iloc[0][11:16].to_numpy(dtype=str),
        prefs.loc[prefs["Email Address"] == email].iloc[0][16:21].to_numpy(dtype=str),
        prefs.loc[prefs["Email Address"] == email].iloc[0][21:26].to_numpy(dtype=str),
        prefs.loc[prefs["Email Address"] == email].iloc[0][26:31].to_numpy(dtype=str),
    ]

    # Remove numbers
    no_numbers_titles = []
    for session_titles in choices:
        no_numbers_titles.append(remove_numbers(session_titles))

    for i in np.arange(5):
        score += check_sessions(sessions[i], no_numbers_titles[i])

    return score


def calculate_scores(emails, final_choices_table):
    scores = np.empty(0)
    for email in emails:
        score = one_score(email, final_choices_table)
        scores = np.append(scores, score)
    return scores


In [ ]:
# def count_alumni(schedules):
#     for x in np.arange(0,schedules.num_rows):
#         person = schedules.row(x)
#         if "Alumni Panel (TBD)" in person:
#             choices = np.array([person.item("Session One"), person.item("Session Two"), person.item("Session Three"), person.item("Session Four"), person.item("Session Five")])
#             grouped_choices = Table().with_columns("Choices", choices).group("Choices")
#             amount_of_alumni = grouped_choices.where("Choices", "Alumni Panel (TBD)").row(0).item(1)
#             if amount_of_alumni > 1:
#                 print(f"{person.item('Email')} got the Career Panel {amount_of_alumni} times")


In [ ]:
def individual_schedules_generate(schedule, save_location, file_name, full=True):
    document = Document()

    styles = document.styles
    section = document.sections[0]
    header = section.header

    header_style = styles.add_style("MyHeader", WD_STYLE_TYPE.PARAGRAPH)
    header_style_font = header_style.font
    header_style_font.name = "Fulbo Argenta"
    header_style_font.bold = True
    header_style_font.size = Pt(11)

    fun_header = styles.add_style("FunHeader", WD_STYLE_TYPE.CHARACTER)
    fun_header_font = fun_header.font
    fun_header_font.name = "Fulbo Argenta"
    fun_header_font.italic = True
    fun_header_font.size = Pt(8)

    name_style = styles.add_style("Name", WD_STYLE_TYPE.PARAGRAPH)
    name_font = name_style.font
    name_font.name = "Georgia"
    name_font.bold = True
    name_font.size = Pt(25)

    sub_style = styles.add_style("SubHeader", WD_STYLE_TYPE.PARAGRAPH)
    sub_style_font = sub_style.font
    sub_style_font.name = "Georgia"
    sub_style_font.underline = True
    sub_style_font.size = Pt(12)

    sub_text = styles.add_style("SubText", WD_STYLE_TYPE.CHARACTER)
    sub_text_font = sub_text.font
    sub_text.name = "Georgia"
    sub_text_font.bold = True
    sub_text_font.italic = True
    sub_text_font.underline = False
    sub_style.font.size = Pt(12)

    table_header_style = styles.add_style("TableHeader", WD_STYLE_TYPE.PARAGRAPH)
    table_header_font = table_header_style.font
    table_header_font.name = "Times New Roman"
    table_header_font.size = Pt(15)
    table_header_font.bold = True

    def create_stud_schedule(name, advisory, grade, stud_schedule):
        # Create Phoenix Fest Header at the top of the document
        my_header = header.paragraphs[0]
        my_header.text = "PHOENIX FEST 2023 SCHEDULE"
        my_header.style = styles["MyHeader"]
        header_run = my_header.add_run(
            "\nMade with \U00002764  by Arnav S.", styles["FunHeader"]
        )
        my_header.alignment = WD_ALIGN_PARAGRAPH.RIGHT

        # Student's name
        name_paragraph = document.add_paragraph(name, styles["Name"])

        # Student's advisory
        advisory_paragraph = document.add_paragraph("Advisory", styles["SubHeader"])
        advisory_run = advisory_paragraph.add_run(f": {advisory}", styles["SubText"])

        # Student's Grade
        grade_paragraph = document.add_paragraph("Grade", styles["SubHeader"])
        grade_run = grade_paragraph.add_run(f": {str(grade)}", styles["SubText"])

        # Initialize Table
        table = document.add_table(rows=1, cols=3)
        table.alignment = WD_TABLE_ALIGNMENT.CENTER
        table.style = "Table Grid"

        # Create & Style Header Cells
        hdr_cells = table.rows[0].cells
        hdrs = ("Block", "Presentation Title", "Room # / Teacher")
        for i in range(0, 3):
            p = hdr_cells[i].add_paragraph(hdrs[i])
            p.style = styles["TableHeader"]
            p.alignment = WD_ALIGN_PARAGRAPH.CENTER

        # Add Student's Specific Information to the Schedule
        for block, pres, room in stud_schedule:
            row_cells = table.add_row().cells
            row_cells[0].text = block
            p2 = row_cells[1].add_paragraph(pres)
            p2.alignment = WD_ALIGN_PARAGRAPH.CENTER
            p3 = row_cells[2].add_paragraph(room)
            p3.alignment = WD_ALIGN_PARAGRAPH.CENTER

        if full:
            document.add_page_break()

    for i in np.arange(0, len(schedule.index)):
        student_schedule = schedule.iloc[i]
        student_name = (
            f"{student_schedule['Last Name']}, {student_schedule['First Name']}"
        )
        student_advisory = student_schedule["Advisor"]
        student_grade = student_schedule["Grade Level"]
        student_records = (
            (
                "1. Academics \n9:00 - 9:35",
                student_schedule["Session One"],
                f"{int(student_schedule['Session One Room #'])} / {student_schedule['Session One Room Location']}",
            ),
            (
                "2. Culture \n9:40 - 10:15",
                student_schedule["Session Two"],
                f"{int(student_schedule['Session Two Room #'])} / {student_schedule['Session Two Room Location']}",
            ),
            (
                "3. FEST \n10:20 - 10:55",
                student_schedule["Session Three"],
                f"{int(student_schedule['Session Three Room #'])} / {student_schedule['Session Three Room Location']}",
            ),
            (
                "4. Community \n11:00 - 11:35",
                student_schedule["Session Four"],
                f"{int(student_schedule['Session Four Room #'])} / {student_schedule['Session Four Room Location']}",
            ),
            (
                "5. Experience \n11:40 – 12:15",
                student_schedule["Session Five"],
                f"{int(student_schedule['Session Five Room #'])} / {student_schedule['Session Five Room Location']}",
            ),
        )
        create_stud_schedule(
            student_name, student_advisory, student_grade, student_records
        )
    filepath = save_location + f"/{file_name}.docx"
    document.save(filepath)


In [ ]:
def rosters_schedule_generate(schedule, save_location, file_name):
    document = Document()

    styles = document.styles
    section = document.sections[0]
    header = section.header

    header_style = styles.add_style("MyHeader", WD_STYLE_TYPE.PARAGRAPH)
    header_style_font = header_style.font
    header_style_font.name = "Fulbo Argenta"
    header_style_font.bold = True
    header_style_font.size = Pt(11)

    fun_header = styles.add_style("FunHeader", WD_STYLE_TYPE.CHARACTER)
    fun_header_font = fun_header.font
    fun_header_font.name = "Fulbo Argenta"
    fun_header_font.italic = True
    fun_header_font.size = Pt(8)

    name_style = styles.add_style("Name", WD_STYLE_TYPE.PARAGRAPH)
    name_font = name_style.font
    name_font.name = "Georgia"
    name_font.bold = True
    name_font.size = Pt(25)

    sub_style = styles.add_style("SubHeader", WD_STYLE_TYPE.PARAGRAPH)
    sub_style_font = sub_style.font
    sub_style_font.name = "Georgia"
    sub_style_font.underline = True
    sub_style_font.size = Pt(13)

    sub_text = styles.add_style("SubText", WD_STYLE_TYPE.CHARACTER)
    sub_text_font = sub_text.font
    sub_text.name = "Georgia"
    sub_text_font.bold = True
    sub_text_font.italic = True
    sub_text_font.underline = False
    sub_style.font.size = Pt(13)

    table_header_style = styles.add_style("TableHeader", WD_STYLE_TYPE.PARAGRAPH)
    table_header_font = table_header_style.font
    table_header_font.name = "Times New Roman"
    table_header_font.size = Pt(20)
    table_header_font.bold = True

    person_style = styles.add_style("PersonStyle", WD_STYLE_TYPE.PARAGRAPH)
    person_style_font = person_style.font
    person_style_font.name = "Georgia"
    table_header_font.size = Pt(13)

    def create_roster_doc(
        title, supervisor, room, room_number, category, first_names, last_names
    ):
        # Create Phoenix Fest Header at the top of the document
        my_header = header.paragraphs[0]
        my_header.text = "PHOENIX FEST 2023 ROSTER"
        my_header.style = styles["MyHeader"]
        header_run = my_header.add_run(
            "\nMade with \U00002764  by Arnav S.", styles["FunHeader"]
        )
        my_header.alignment = WD_ALIGN_PARAGRAPH.RIGHT

        # Session Title
        heading_paragraph = document.add_paragraph(title, styles["Name"])
        heading_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

        # Intialize Info Table 1
        info_table = document.add_table(rows=1, cols=3)
        info_table.alignment = WD_TABLE_ALIGNMENT.CENTER
        info_cells = info_table.rows[0].cells

        # Supervisor (confusing title, but typically the )
        supervisor_paragraph = info_cells[0].add_paragraph(
            "Supervisor", styles["SubHeader"]
        )
        supervisor_run = supervisor_paragraph.add_run(
            f": {supervisor}", styles["SubText"]
        )
        supervisor_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

        # Room Number
        room_paragraph = info_cells[1].add_paragraph(
            "Room Location", styles["SubHeader"]
        )
        room_run = room_paragraph.add_run(
            f": {room} ({room_number})", styles["SubText"]
        )
        room_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

        # Category
        category_paragraph = info_cells[2].add_paragraph(
            "Category", styles["SubHeader"]
        )
        category_run = category_paragraph.add_run(f": {category}", styles["SubText"])
        category_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

        # People's Table
        people_table = document.add_table(rows=1, cols=4)
        people_table.alignment = WD_TABLE_ALIGNMENT.CENTER
        people_table.style = "Table Grid"
        people_table.style.paragraph_format.line_spacing_rule = WD_LINE_SPACING.EXACTLY

        # Create & Style People's Table Header Cells
        hdr_cells = people_table.rows[0].cells
        hdrs = ("Student Number", "First Name", "Last Name", "Attendance")
        for i in range(0, 4):
            p = hdr_cells[i].add_paragraph(hdrs[i])
            p.style = styles["TableHeader"]
            p.alignment = WD_ALIGN_PARAGRAPH.CENTER

        x = 1
        for first, last in zip(first_names, last_names):
            row_cells = people_table.add_row().cells
            p1 = row_cells[0].add_paragraph(str(x), styles["PersonStyle"])
            p1.alignment = WD_ALIGN_PARAGRAPH.CENTER
            p2 = row_cells[1].add_paragraph(first, styles["PersonStyle"])
            p2.alignment = WD_ALIGN_PARAGRAPH.CENTER
            p3 = row_cells[2].add_paragraph(last, styles["PersonStyle"])
            p3.alignment = WD_ALIGN_PARAGRAPH.CENTER
            p4 = row_cells[3].add_paragraph("\U00002610")
            p4.alignment = WD_ALIGN_PARAGRAPH.CENTER
            x += 1

        document.add_page_break()

    # Sort schedule by advisory name for ease of access
    concated_blocks = []
    for i in range(1, 6):
        concated_blocks.append(
            schedule.loc[schedule["Category"] == str(i)].sort_values(by="Supervisor")
        )
    schedule = pd.concat(concated_blocks)

    for i in np.arange(0, len(schedule.index)):
        session = schedule.iloc[i]
        title = session["Program Title"]
        supervisor = session["Supervisor"]
        room = session["Room Location"]
        room_number = int(session["Room #"])
        category = int(session["Category"])
        data = {"Email": session["People"]}
        names = pd.DataFrame(data)
        names = pd.merge(names, roster, left_on="Email", right_on="Email").sort_values(
            by="Last Name"
        )[["First Name", "Last Name"]]
        first_names = names["First Name"]
        last_names = names["Last Name"]
        create_roster_doc(
            title, supervisor, room, room_number, category, first_names, last_names
        )

    filepath = save_location + f"/{file_name}.docx"
    document.save(filepath)


In [ ]:
def send_email(emails, schedule):
    for email in emails:
        student_schedule = schedule.loc[schedule["Email"] == email]
        first_name = student_schedule["First Name"].item()
        last_name = student_schedule["Last Name"].item()
        student_name = f"{last_name}, {first_name}"
        individual_schedules_generate(
            student_schedule,
            "/work/2022-2023/Exported Files/Docs/Individual Schedules",
            student_name,
            False,
        )
        sleep(0.1)

        msg = MIMEMultipart("alternative")
        msg["Subject"] = f"Phoenix Fest Schedule for {student_name}"
        msg["From"] = "Phoenix Fest Committee"
        msg["To"] = "asareen@raleighcharterhs.org"

        text = """\ 
            Here is your schedule for Phoenix Fest, which will be held from 9:00-12:15 on Friday, May 19th. 
            Be sure to follow your assigned schedule, as attendance will be taken in each session. 
            If you notice an error, please see Ms. Arcaro in room 230 today.
            We're looking forward to seeing you on our fun-filled Friday! 
        """
        html_text = (
            """\
        <html>
            <body>
                <p>Hi """
            + first_name
            + """,</p>
                <p>
                Here is your schedule for Phoenix Fest, which will be held from 9:00-12:15 on Friday, May 19th. 
                Be sure to follow your assigned schedule, as attendance will be taken in each session. 
                <b>If you notice an error, please see Ms. Arcaro in room 230 today.</b> 
                We're looking forward to seeing you on our fun-filled Friday! 
                </p>
                <p>Take care, <br>
                Phoenix Fest Committee </p>
        </body>
    </html> """
        )

        no_html = MIMEText(text, "plain")
        html = MIMEText(html_text, "html")

        msg.attach(no_html)
        msg.attach(html)

        filename = f"/work/2022-2023/Exported Files/Docs/Individual Schedules/{last_name}, {first_name}.docx"
        with open(filename, "rb") as f:
            attachment = MIMEApplication(f.read(), Name=basename(filename))
            attachment[
                "Content-Disposition"
            ] = f"attachment; filename={basename(filename)}"

        msg.attach(attachment)

        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(os.environ.get("EMAIL"), os.environ.get("PASSWORD"))
            server.sendmail(
                "asareen@raleighcharterhs.org",
                "asareen@raleighcharterhs.org",
                msg.as_string(),
            )
            print(f"Email Sent to {student_name}!")


In [ ]:
def main():
    # Create schedules for all five sessions!
    one = session_one()
    two = session_two()
    three = session_three()
    four = session_four()
    five = session_five()
    emails = one[1].sort_values(by="People")["People"].to_numpy(dtype=str)

    # The following lines consolidate all of the names, room numbers and room locations of the programs, based on the individual schedules of students
    program_one = one[1].sort_values(by="People")["Program"].to_numpy(dtype=str)
    program_one_room_numbers = one[1].sort_values(by="People")["Room #"].to_numpy()
    program_one_room_locations = (
        one[1].sort_values(by="People")["Room Location"].to_numpy()
    )

    program_two = two[1].sort_values(by="People")["Program"].to_numpy(dtype=str)
    program_two_room_numbers = two[1].sort_values(by="People")["Room #"].to_numpy()
    program_two_room_locations = (
        two[1].sort_values(by="People")["Room Location"].to_numpy()
    )

    program_three = three[1].sort_values(by="People")["Program"].to_numpy(dtype=str)
    program_three_room_numbers = three[1].sort_values(by="People")["Room #"].to_numpy()
    program_three_room_locations = (
        three[1].sort_values(by="People")["Room Location"].to_numpy()
    )

    program_four = four[1].sort_values(by="People")["Program"].to_numpy(dtype=str)
    program_four_room_numbers = four[1].sort_values(by="People")["Room #"].to_numpy()
    program_four_room_locations = (
        four[1].sort_values(by="People")["Room Location"].to_numpy()
    )

    program_five = five[1].sort_values(by="People")["Program"].to_numpy(dtype=str)
    program_five_room_numbers = five[1].sort_values(by="People")["Room #"].to_numpy()
    program_five_room_locations = (
        five[1].sort_values(by="People")["Room Location"].to_numpy()
    )

    # Create the initial schedules for each student
    data = {
        "Email": emails,
        "Session One": program_one,
        "Session One Room #": program_one_room_numbers,
        "Session One Room Location": program_one_room_locations,
        "Session Two": program_two,
        "Session Two Room #": program_two_room_numbers,
        "Session Two Room Location": program_two_room_locations,
        "Session Three": program_three,
        "Session Three Room #": program_three_room_numbers,
        "Session Three Room Location": program_three_room_locations,
        "Session Four": program_four,
        "Session Four Room #": program_four_room_numbers,
        "Session Four Room Location": program_four_room_locations,
        "Session Five": program_five,
        "Session Five Room #": program_five_room_numbers,
        "Session Five Room Location": program_five_room_locations,
    }

    final_choices = pd.DataFrame(data)

    # Combine official RCHS Data with our schedules
    final_choices = pd.merge(final_choices, roster, left_on="Email", right_on="Email")

    # Rearrange columns
    final_choices = final_choices[
        [
            "Last Name",
            "First Name",
            "Advisor",
            "Grade Level",
            "Email",
            "Session One",
            "Session One Room #",
            "Session One Room Location",
            "Session Two",
            "Session Two Room #",
            "Session Two Room Location",
            "Session Three",
            "Session Three Room #",
            "Session Three Room Location",
            "Session Four",
            "Session Four Room #",
            "Session Four Room Location",
            "Session Five",
            "Session Five Room #",
            "Session Five Room Location",
        ]
    ]

    # Calculate scores
    scores = calculate_scores(final_choices["Email"], final_choices)
    final_choices["Score"] = scores

    # Combine all program sessions into one table
    sessions_table = combine_session(one[0], two[0], three[0], four[0], five[0])

    return sessions_table, final_choices.sort_values(by="Last Name")


In [ ]:
phoenix_fest = main()
print(f"Average score of this run is {np.average(phoenix_fest[1]['Score'])}")
phoenix_fest[0]


In [ ]:
phoenix_fest[1]


In [ ]:
def main_export():
    phoenix_fest[0].to_csv(f"Exported Files/Sessions - {datetime.now()}.csv")
    phoenix_fest[1].to_csv(f"Exported Files/Individual Schedules - {datetime.now()}.csv")
    rosters_schedule_generate(phoenix_fest[0], "/work/2022-2023/Exported Files/Docs", f"Roster - {datetime.now()}")
    individual_schedules_generate(phoenix_fest[1], "/work/2022-2023/Exported Files/Docs", f"Schedule - {datetime.now()}")

In [ ]:
# send_email(phoenix_fest[1]["Email"].to_numpy(dtype=str), phoenix_fest[1])

In [ ]:
# best_score = 30
# best_sessions = None
# best_choices = None

# for i in np.arange(0,50):
#     try:
#         run = main()
#     except:
#         pass
#     else:
#         average_score = np.average(run[1]["Score"].to_numpy())
#         if average_score < best_score:
#             best_score = average_score
#             best_sessions = run[0]
#             best_choices = run[1]

# print(f"The Average score is: {np.average(best_choices['Score'].to_numpy())}")
# phoenix_fest = (best_sessions, best_choices)

lowest_spots_left = 15
best_sessions = None
best_choices = None

for i in np.arange(0,1000):
    try:
        run = main()
    except:
        pass
    else:
        session_spots_left = np.average(list(map(float,list((run[0].sort_values(by="Spots Left", ascending=False).iloc[:6]["Spots Left"].to_numpy())))))
        if lowest_spots_left > session_spots_left:
            print(f"{i}: {session_spots_left}")
            lowest_spots_left = session_spots_left
            best_sessions = run[0]
            best_choices = run[1]

print(list(map(float,list((best_sessions.sort_values(by='Spots Left', ascending=False).iloc[:6]['Spots Left'].to_numpy())))))
print(f"Minimum Average: {np.average(list(map(float,list((best_sessions.sort_values(by='Spots Left').head(6)['Spots Left'].to_numpy())))))}")
print(f"The Average score is: {np.average(best_choices['Score'].to_numpy())}")
phoenix_fest = (best_sessions, best_choices)

In [ ]:
phoenix_fest[0]

In [ ]:
phoenix_fest[1]

In [ ]:
main_export()

# Notes for Version 6

- Implement SMTP

- Rework scheduling process for "unpopular" sessions
    - First, identify a threshold in which the total of number of signs-ups *does not* yield a sufficient number of presentees in a session (this in its self will need to be a threshold but is rather arbitrary; for this year, ten seems like a good number). Assign this number to *t<sub>1</sub>*
        - Quick analysis indicates *t<sub>1</sub>* is likely around 25-35
    - Then, using *t<sub>1</sub>*, find all the sessions that do not meet this threshold **and** do not meet another threshold, *t<sub>2</sub>*, which is the total number of sign ups for a specific choice (i.e. first, second, third...)
        - Quick analysis here indicates that *t<sub>2</sub>* is like around 4-8
    - If a session meets *both* requirements, then assign them special privieliges in assigning; we can call this designation **FA** (force assign)
- For all FA sessions, take all the people who signed up for the session, regardless if it was their third, fourth, or fifth choice, and assign them to the session
    - To do this, we can introduce another parameter into `session_assign()` that checks if a session meets this special condition, and check for this special condition within `schedule()` 
        - This should be sufficient, given that this only needs to happen once per block


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a723c23f-b8da-4d0b-84c5-26f14cea2568' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>